In [ ]:
import sys
from math import log


test_pred1 = [1,0,1,0,1,0,0]
test_pred2 = [1,1,0,1,0]
test_target1 = [0,0,0,1,1,0,0]
test_target2 = [0,0,1,1,0]


# cross entropy
def ce(pred, targ):

    pred = [float(p) for p in pred]
    targ = [float(t) for t in targ]
    
    # Calculate cross-entropy loss
    loss = -sum(t * log(max(p, 1e-6)) + (1 - t) * log(max(1 - p, 1e-6)) for p, t in zip(pred, targ))
    
    return loss / len(pred)

# dice loss
def dl(pred, targ):
    pred = [float(p) for p in pred]
    targ = [float(t) for t in targ]
    
    # Calculate Dice coefficient
    intersection = sum(p * t for p, t in zip(pred, targ))
    dice_coeff = (2 * intersection + 1e-6) / (sum(pred) + sum(targ) + 1e-6)
    
    # Calculate Dice loss
    loss = 1 - dice_coeff
    
    return loss

print("Cross-Entropy Loss for test_pred1 and test_target1:", ce(test_pred1, test_target1))
print("Cross-Entropy Loss for test_pred2 and test_target2:", ce(test_pred2, test_target2))
print("Dice Loss for test_pred1 and test_target1:", dl(test_pred1, test_target1))
print("Dice Loss for test_pred2 and test_target2:", dl(test_pred2, test_target2))

Cross-Entropy Loss for test_pred1 and test_target1: 5.920933667691045
Cross-Entropy Loss for test_pred2 and test_target2: 8.289306734767262
Dice Loss for test_pred1 and test_target1: 0.5999998800000239
Dice Loss for test_pred2 and test_target2: 0.5999998800000239


In [ ]:
import torch
import torch.nn.functional as F

# Example tensors
pred = torch.randn(1, 3, 4, 4, requires_grad=True)  
target = torch.randint(0, 3, (1, 4, 4)) 
print(pred)
print(pred.shape)
print(target)
print(target.shape)

# Use built-in ignore_index
loss = F.cross_entropy(pred, target) # , ignore_index=255
print(loss)
print('############')
target[0, 0, 0] = 255  # Let's say 255 is the ignore index
print(pred)
print(pred.shape)
print(target)
print(target.shape)
loss = F.cross_entropy(pred, target, ignore_index=255)
print(loss)

tensor([[[[-0.4372,  0.5410, -0.7114,  0.0496],
          [-1.0471, -0.3032, -2.3385,  0.4909],
          [-1.2547, -0.7022,  1.1577,  1.1411],
          [-1.1810,  1.0437, -0.4040, -2.3348]],

         [[-0.8528, -1.5593, -1.0323,  0.6387],
          [-0.7896,  0.7512, -0.3808,  0.1616],
          [ 0.6100, -0.1642, -0.7026,  1.7131],
          [-1.3847,  0.5318, -0.1479,  0.0535]],

         [[-0.3831,  0.2597,  0.2442,  0.2699],
          [ 1.1634, -2.6126, -0.5483,  0.5152],
          [-1.2074,  1.3672,  0.8899,  0.8036],
          [-0.4986, -0.4400,  0.0515, -2.6332]]]], requires_grad=True)
torch.Size([1, 3, 4, 4])
tensor([[[1, 0, 1, 0],
         [0, 1, 2, 0],
         [1, 0, 2, 0],
         [2, 0, 1, 2]]])
torch.Size([1, 4, 4])
tensor(1.2403, grad_fn=<NllLoss2DBackward0>)
############
tensor([[[[-0.4372,  0.5410, -0.7114,  0.0496],
          [-1.0471, -0.3032, -2.3385,  0.4909],
          [-1.2547, -0.7022,  1.1577,  1.1411],
          [-1.1810,  1.0437, -0.4040, -2.3348]],

    

### counts the number of pixels for each class

In [7]:
import os
import pandas as pd
from PIL import Image
import numpy as np

# Path to the directory containing .png files
annotation_dir = "I:\Datasets\Hierarchical Datasets\TL_pano\processed\labels"

# Path to the .csv file containing pixel values and class mappings
csv_file = "I:\Datasets\Hierarchical Datasets\TL_pano\processed\class_map.csv"

# Load the class mapping from the .csv file
class_mapping = pd.read_csv(csv_file)
print(class_mapping)

# Initialize a dictionary to store pixel counts for each class
pixel_counts = {row['class_name']: 0 for _, row in class_mapping.iterrows()}
num_imgs = 0
# Traverse and process each .png file in the directory
for file_name in os.listdir(annotation_dir):
    if file_name.endswith(".png"):
        num_imgs += 1
        file_path = os.path.join(annotation_dir, file_name)
        
        # Load the image
        image = Image.open(file_path)
        image_array = np.array(image)
        
        # Count pixels for each class
        for _, row in class_mapping.iterrows():
            class_value = row['pixel_val']
            pixel_counts[row['class_name']] += np.sum(image_array == class_value)
print('############################')
# Print the pixel counts for each class
print("Pixel counts for each class:")
for class_name, count in pixel_counts.items():
    print(f"{class_name}: {count/num_imgs}")

   class_id  class_name  pixel_val
0         0  background        0.0
1         1       upper      212.0
2         2       lower      255.0
3         3       tooth        NaN
4         4        pulp      127.0
5         5      dentin      170.0
6         6      enamel       85.0
7         7   composite       42.0
############################
Pixel counts for each class:
background: 5161360.47715736
upper: 97187.13197969543
lower: 159645.91370558375
tooth: 0.0
pulp: 99224.96446700508
dentin: 581722.0203045686
enamel: 147300.38578680204
composite: 38744.918781725886


In [ ]:
from torch import randint, tensor
import torchmetrics
target = randint(0, 2, (10, 25, 25))
pred = randint(0, 2, (10, 25, 25))
pred = tensor(pred)
target = tensor(target)
print(pred)
print(pred.shape)
print(target)
print(target.shape)

jaccard = torchmetrics.JaccardIndex(task="multiclass", num_classes=2)
jaccard(pred, target)

tensor([[[1, 1, 1,  ..., 0, 1, 0],
         [1, 1, 0,  ..., 0, 1, 0],
         [1, 0, 1,  ..., 0, 0, 1],
         ...,
         [0, 0, 1,  ..., 1, 0, 0],
         [0, 0, 1,  ..., 1, 0, 0],
         [1, 0, 1,  ..., 1, 0, 1]],

        [[1, 1, 1,  ..., 0, 1, 0],
         [0, 1, 1,  ..., 1, 0, 1],
         [0, 1, 1,  ..., 0, 1, 1],
         ...,
         [1, 0, 0,  ..., 0, 0, 0],
         [0, 1, 0,  ..., 0, 1, 0],
         [0, 0, 1,  ..., 0, 1, 0]],

        [[0, 0, 1,  ..., 1, 0, 1],
         [1, 1, 1,  ..., 1, 1, 1],
         [1, 1, 1,  ..., 1, 1, 1],
         ...,
         [0, 1, 0,  ..., 0, 0, 1],
         [1, 0, 0,  ..., 1, 0, 1],
         [0, 0, 1,  ..., 0, 0, 1]],

        ...,

        [[1, 1, 0,  ..., 1, 1, 0],
         [1, 1, 1,  ..., 1, 1, 1],
         [0, 0, 0,  ..., 1, 1, 1],
         ...,
         [0, 0, 0,  ..., 0, 1, 1],
         [1, 1, 1,  ..., 1, 1, 1],
         [1, 1, 1,  ..., 1, 1, 0]],

        [[1, 0, 1,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 1, 0],
         [1,

C:\Users\Banksy\AppData\Local\Temp\ipykernel_6852\2990729996.py:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  pred = tensor(pred)
C:\Users\Banksy\AppData\Local\Temp\ipykernel_6852\2990729996.py:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  target = tensor(target)


tensor(0.3376)